# Speech Recognition using Graphs

Team members: Adrian Löwenstein, Kiran Bacsa, Manuel Vonlanthen<br>
Date:         22.01.2018

## Problem Description

Exactly describe what we wanted to do, i.e. classify 20 out of 30 words, the other 10 as unknown...<br>
List the words...<br>
Discuss where the data is from...<br>
Mention the difference between the original kaggle competition to what whe did, e.g. ignore silence, not using thensor flow, only work with training data, to not be dependent on the kaggle competition...<br>

## Imports

## Recompute

**WARNING** If you set recompute to True this will reextract all featrues and re-classify all audio file, which will take several days, so do not do it. It is here for completeness so you can see how the steps were done during th project. We've already computed these steps and saved the results into pickle files. Our entire used data, as well as the pickle files can be found on "Link".

In [3]:
recompute = False

## Feature Extraction (Adrian)

Describe the data set....<br>
Describe the entire pipeline from audio file to feature vector for one audio dile...<br>
Shortly mentione other Features that were tried...<br>
Set up python function in which the features of the entire training set could be extracted and put it into a if recompute is true... <br>
Load the pickle with all features in them,...

## Spectral clustering

Our first approach was to naively reuse the code from assignment 3 : spectral graph theory. By combining all our of samples into
a single graph and extracting the resulting graph laplacian, we hope to identify clusters which would correspond to the different words that need to be classified.

**WARNING** set indexes for features_og instead of taking entire dataset: 

Our best results were obtained using the cosine distance as our kernel metric.

In [ ]:
# get and normalize mfccs
features = pd.DataFrame(features_og['mfcc'])
features -= features.mean(axis=0)
features /= features.std(axis=0)

# compute spatial distance
distances = spatial.distance.squareform(spatial.distance.pdist(features,'cosine'))

# generate weights with RBF kernel
n=distances.shape[0]
kernel_width = distances.mean()
weights = np.exp(np.divide(-np.square(distances),kernel_width**2))
np.fill_diagonal(weights,0)

Unfortunatly, increasing sparsity has the effect of reducing classification accuracy. We therefore decided to remove k-NN
sparsification all together and keep the sample graph as it is.

In [ ]:
# compute laplacian
degrees = np.sum(weights,axis=0)
laplacian = np.diag(degrees**-0.5) @ (np.diag(degrees) - weights) @ np.diag(degrees**-0.5)
laplacian = sparse.csr_matrix(laplacian)
plt.spy(laplacian.todense());

We can now calculate the eigenvectors of the Laplacian matrix. These eigenvectors will be used as feature vectors for our 
classifier.

In [ ]:
eigenvalues, eigenvectors = sparse.linalg.eigsh(A=laplacian,k=25,which='SM')
plt.plot(eigenvalues[1:], '.-', markersize=15);

In [ ]:
fix, axes = plt.subplots(5, 5, figsize=(17, 8))
for i in range(1,6):
    for j in range(1,6):
        x = eigenvectors[:,i]
        y = eigenvectors[:,j]
        labels = np.sign(x)
        axes[i-1,j-1].scatter(x, y, c=labels, cmap='RdBu', alpha=0.5)

Before moving on to the classification phase, we first need to split our samples into a training and a test set. In order to 
prevent our classifier from being biased by the training set, we made sure that both sets have an equal proportion of classes 
(i.e. words).

In [ ]:
# sample classes such as test and train have equal proportion of classes
def split_sample_equal(label_vec, n_classes, train_size, test_size):

    train_x = np.array([])
    train_y = np.array([])

    test_x = np.array([])
    test_y = np.array([])

    for i in range(n_classes):
        class_index = np.where(label_vec == (i+1))[0]
        random_index = np.random.choice(range(len(class_index)), size=train_size+test_size, replace=False)

        train_x_class = class_index[random_index[:train_size]]
        train_y_class = label_vec[train_x_class]
        train_x = np.append(train_x, train_x_class)
        train_y = np.append(train_y, train_y_class)

        test_x_class = class_index[random_index[train_size:train_size+test_size]]
        test_y_class = label_vec[test_x_class]
        test_x = np.append(test_x, test_x_class)
        test_y = np.append(test_y, test_y_class)
        
    return train_x, train_y, test_x, test_y
    

In [ ]:
train_x, train_y, test_x, test_y = split_sample_equal(label_vec, len(class_names), 80, 20)

In [ ]:
# get features
train_features = eigenvectors[train_x.astype(int),:]
test_features = eigenvectors[test_x.astype(int),:]

A wide range of classifiers were tested on our input features. Remarkably, a very simple classifier such as the [Gaussian Naive Bayes classifier](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) produced far better results than more advanced techniques. This is mainly because the graph datapoints were generated using a gaussian kernel, and is therefore sensible to assume that our feature distribution will be gaussian as well. However, the best results were obtained using a [Quadratic Discriminant Analysis classifier](https://en.wikipedia.org/wiki/Quadratic_classifier).

In [ ]:
def fit_and_test(clf, train_x, train_y, test_x, test_y):
    clf.fit(train_x, train_y)  
    predict_y = clf.predict(test_x)
    print('accuracy : ', np.sum(test_y==predict_y)/len(test_y))
    return predict_y

In [ ]:
clf = QuadraticDiscriminantAnalysis()
predict_y = fit_and_test(clf, train_features, train_y, test_features, test_y)

Once our test set has been classified we can visualize the effectiveness of our classification using a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix).

In [ ]:
def plot_confusion_matrix(test_y, predict_y, class_names):

    conf_mat=confusion_matrix(test_y,predict_y)

    plt.figure(figsize=(10,10))
    plt.imshow(conf_mat/np.sum(conf_mat,axis=1),cmap=plt.cm.hot)
    tick = np.arange(len(class_names))
    plt.xticks(tick, class_names,rotation=90)
    plt.yticks(tick, class_names)
    plt.ylabel('ground truth')
    plt.xlabel('prediction')
    plt.title('Confusion matrix')
    plt.colorbar()

In [ ]:
plot_confusion_matrix(test_y, predict_y, class_names)

Finally we can focus on the core words that need to be classified and label the rest as 'unknown'.

In [ ]:
acc_adapted = np.sum(test_y_adapted==predict_y_adapted)/len(test_y_adapted)
print('accuracy for main words classification : ', acc_adapted)

In [ ]:
plot_confusion_matrix(test_y_adapted,predict_y_adapted, class_names)

In conclusion, we can say that, using spectral clustering, we were able to leverage the properties of graph theory to find 
relevant features in speech recognition. However, the accuracy achieved with our model is too low for any practical applications. Moreover, this model does not benefit from sparsity, meaning that it will not be able to scale with large datasets. 

## Semi-Supervised Classification (Manuel)

I this section we describe the entire semi-supervised classification we used in detail. It is basically a step by step description
of the semisup pipeline using only one test batch of 200 in a Graph of 5000 nodes. <br>

## Method Validation (Kiran)

Describe that we tweaked parameters and which were found to be the optimal ones, comment on it...<br>
Show the resulting accuracy that resulted when going trough the entire training set. For this, simply load a pickle or numpy array, with the results in it and comment on it<br>
Add a .py funtion with which we could theoretically call to recompute the accuracy of the entire training set (in a "if recompute is true" conditioning). Add the function into main_pipeline.py. <br>
(Optional) Also add clustering approach to compare, otherwise we will just mention it...

In [ ]:
train_x, train_y, test_x, test_y = split_sample_equal(label_vec, len(class_names), 160, 100)

I guess these will probably already be in Manuel's part...

### Main loop :

In [ ]:
batch_size = 200
NEIGHBORS = 30
n_batch = int(len(test_x) / batch_size)

alpha = 1e-3
beta = 1e-7

In [ ]:
accuracy_tot = []
accuracy = []
remaining_test = np.array(test_x)

# encode training samples classes into 1-hot array
n_class = np.max(train_y)
Y = np.eye(len(class_names))[train_y - 1].T

for batch in tqdm(range(n_batch)):
    potential_elements  = np.array(list(enumerate(remaining_test)))
    indices = np.random.choice(potential_elements[:,0].reshape(-1,), batch_size, replace=False)
    batch_index = potential_elements[:,0].reshape(-1,)[indices]
    remaining_test = np.delete(remaining_test, indices)
        
    features = pd.DataFrame(features_og['mfcc'], np.append(train_x, batch_index))
    features -= features.mean(axis=0)
    features /= features.std(axis=0)

    # keep mapping from batch id to main id
    id_mapping = dict(zip(features.index, range(len(features))))
    
    distances = spatial.distance.squareform(spatial.distance.pdist(features,'cosine'))

    n=distances.shape[0]
    kernel_width = distances.mean()
    weights = np.exp(np.divide(-np.square(distances),kernel_width**2))
    np.fill_diagonal(weights,0)
    
    for i in range(weights.shape[0]):
        idx = weights[i,:].argsort()[:-NEIGHBORS]
        weights[i,idx] = 0
        weights[idx,i] = 0
        
    degrees = np.sum(weights,axis=0)
    laplacian = np.diag(degrees**-0.5) @ (np.diag(degrees) - weights) @ np.diag(degrees**-0.5)
    laplacian = sparse.csr_matrix(laplacian)

    # add test samples to 1-hot array
    M = np.zeros((len(class_names), len(train_y) + batch_size)) # mask matrix
    M[:len(train_y),:len(train_y)] = 1
    Y_compr = np.concatenate((Y, np.zeros((len(class_names), batch_size))), axis=1)
    y = np.concatenate((train_y,np.zeros((batch_size,))))

    # Solve
    X = solve(Y_compr, M, laplacian, alpha = alpha, beta = beta)

    # Make label vector
    x_hat = np.argmax(X,axis = 0)+np.ones(X[0,:].shape)

    # Unify labels 13-30
    x_hat_adapted = adapt_labels(x_hat)
    true_y = np.concatenate((train_y,label_vec[batch_index]))
    y_adapted = adapt_labels(true_y)
    
    # Only consider unknowns
    accuracy_tot.append(np.sum(x_hat[(len(x_hat)-batch_size):]==true_y[(len(x_hat)-batch_size):])/batch_size) # for all 30 words
    accuracy.append(np.sum(x_hat_adapted[(len(x_hat)-batch_size):]==y_adapted[(len(x_hat)-batch_size):])/batch_size) # only core words
    

In [ ]:
#print(accuracy_tot)
print(accuracy)
print('The achieved overall accuracy was {}%'.format(round(np.sum(accuracy)/len(accuracy)*100)))

## Conclusion : 